In [39]:
import os
from dotenv import load_dotenv
from langchain_community.tools import BraveSearch

load_dotenv()
BRAVESEARCH_API_KEY = os.getenv("BRAVESEARCH_API_KEY")
brv_tool = BraveSearch.from_api_key(api_key=BRAVESEARCH_API_KEY, search_kwargs={"count": 3})

In [40]:
from langchain_community.tools import BraveSearch
import yfinance
import json

def get_ticker(inputs):
    company_name = inputs["company_name"]
    tool = BraveSearch.from_api_key(api_key=BRAVESEARCH_API_KEY, search_kwargs={"count": 3})
    return tool.run(f"ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [41]:
get_ticker({"company_name":"tesla"})

'[{"title": "Tesla, Inc. (TSLA) Stock Price, News, Quote & History - Yahoo Finance", "link": "https://finance.yahoo.com/quote/TSLA/", "snippet": "Find the latest <strong>Tesla, Inc.</strong> (TSLA) stock quote, history, news and other vital information to help you with your stock trading and investing."}, {"title": "Tesla, Inc. (TSLA) Stock Price, News, Quotes, & Historic Data | Nasdaq", "link": "https://www.nasdaq.com/market-activity/stocks/tsla", "snippet": "Discover real-time <strong>Tesla</strong>, Inc. Common Stock (TSLA) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq."}, {"title": "TSLA Stock Price | Tesla Inc. Stock Quote (U.S.: Nasdaq) | MarketWatch", "link": "https://www.marketwatch.com/investing/stock/tsla", "snippet": "TSLA | Complete <strong>Tesla</strong> Inc. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview."}]'

In [43]:
get_income_statement({"ticker":"TSLA"})

'"{\\"1735603200000\\":{\\"Tax Effect Of Unusual Items\\":-136800000.0,\\"Tax Rate For Calcs\\":0.2,\\"Normalized EBITDA\\":15392000000.0,\\"Total Unusual Items\\":-684000000.0,\\"Total Unusual Items Excluding Goodwill\\":-684000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":7130000000.0,\\"Reconciled Depreciation\\":5368000000.0,\\"Reconciled Cost Of Revenue\\":80240000000.0,\\"EBITDA\\":14708000000.0,\\"EBIT\\":9340000000.0,\\"Net Interest Income\\":1219000000.0,\\"Interest Expense\\":350000000.0,\\"Interest Income\\":1569000000.0,\\"Normalized Income\\":7677200000.0,\\"Net Income From Continuing And Discontinued Operation\\":7130000000.0,\\"Total Expenses\\":89930000000.0,\\"Rent Expense Supplemental\\":1003000000.0,\\"Total Operating Income As Reported\\":7076000000.0,\\"Diluted Average Shares\\":null,\\"Basic Average Shares\\":null,\\"Diluted EPS\\":null,\\"Basic EPS\\":null,\\"Diluted NI Availto Com Stockholders\\":7130000000.0,\\"Average Dilution Earnings

In [44]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4o",
#     tools=functions,
# )
# assistant
assistant_id = "asst_KX98Kd6DS7FHWmbrHl30Tsod"

In [45]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if the Salesforce stock is a good buy"
)

In [46]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not."
)
run

Run(id='run_njeEMMEtnfWT3AxEArYAisBZ', assistant_id='asst_KX98Kd6DS7FHWmbrHl30Tsod', cancelled_at=None, completed_at=None, created_at=1740736008, expires_at=1740736608, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_ZEN30V6DgFqYgVGgktgZyfM9', function=Function(arguments='{"company_name":"Salesforce"}', name='get_ticker'), type='function')]), type='submit_tool_outputs'), response_format='auto', started_at=1740736010, status='requires_action', thread_id='thread_xQ4oODxEsimd0GCu3Yy6iIR0', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='

In [47]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [ ]:
submit_tool_outputs(run.id, thread.id)

In [49]:
get_run(run.id, thread.id).status

'requires_action'

In [48]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy


In [ ]:
send_message(thread.id, "please go ahead")